# Trabalhando com data e hora no Pandas DataFrame

Data e hora é um tipo de dados comum em projetos de ciência de dados. Freqüentemente, você trabalhará com ele e terá problemas. Descobri que o Pandas é uma biblioteca incrível que contém recursos e recursos abrangentes para trabalhar com data e hora.

Neste artigo, abordaremos os seguintes problemas comuns de data e hora e devemos ajudá-lo a iniciar a análise de dados.

1. Converter strings para data e hora
2. Reúna uma data e hora de várias colunas
3. Obtenha ano, mês e dia
4. Obtenha a semana do ano, o dia da semana e o ano bissexto
5. Obtenha a idade a partir da data de nascimento
6. Melhore o desempenho definindo a coluna de data como o índice
7. Selecione os dados com um ano específico e execute a agregação
8. Selecione os dados com um mês específico e um dia específico do mês
9. Selecione dados entre duas datas
10. Lidar com valores ausentes

# 1. Converta strings para datetime

O Pandas possui uma função interna chamada to_datetime()que pode ser usada para converter strings em data e hora. Vamos dar uma olhada em alguns exemplos

# Com argumentos padrão

O Pandas to_datetime()é capaz de analisar qualquer string de data válida para datetime sem quaisquer argumentos adicionais. Por exemplo:

In [27]:
# Importando a biblioteca
import pandas as pd

# Criando um dataframe
df = pd.DataFrame({'date': ['3/10/2000', '3/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'])
df

,date,value
0,2000-03-10,2
1,2000-03-11,3
2,2000-03-12,4


Por padrão, to_datetime() analisará a string com o formato do mês primeiro ( MM / DD , MM DD ou MM-DD ), e esse arranjo é relativamente único nos Estados Unidos.

Na maior parte do resto do mundo, o dia é escrito primeiro ( DD / MM , DD MM ou DD-MM ). Se quiser que o Pandas considere primeiro o dia em vez do mês, você pode definir o argumento dayfirstcomo True.

In [28]:
df = pd.DataFrame({'date': ['3/10/2000', '3/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df

,date,value
0,2000-10-03,2
1,2000-11-03,3
2,2000-12-03,4


# Formato personalizado

Por padrão, as strings são analisadas usando o analisador integrado do Pandas em dateutil.parser.parse. Às vezes, suas strings podem estar em um formato personalizado, por exemplo, AAAA-DD-MM HH: MM: SS . Pandas to_datetime()tem um argumento chamado formatque permite que você passe um formato personalizado:

In [29]:
df = pd.DataFrame({'date': ['2016-6-10 20:30:0', 
                            '2016-7-1 19:45:30', 
                            '2013-10-12 4:5:1'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'], format="%Y-%d-%m %H:%M:%S")
df

,date,value
0,2016-10-06 20:30:00,2
1,2016-01-07 19:45:30,3
2,2013-12-10 04:05:01,4


A passagem infer_datetime_format=Truepode frequentemente acelerar uma análise se não for um formato ISO8601 exatamente, mas em um formato regular. De acordo com [1], em alguns casos, isso pode aumentar a velocidade de análise em 5–10x.

In [30]:
# Make up 3000 rows
df = pd.DataFrame({'date': ['3/11/2000', '3/12/2000', '3/13/2000'] * 1000 })
%timeit pd.to_datetime(df['date'], infer_datetime_format=True)

1.41 ms ± 46.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%timeit pd.to_datetime(df['date'], infer_datetime_format=False)

1.31 ms ± 30.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Você acabará com um TypeError se a string de data não atender ao formato de carimbo de data / hora.

In [32]:
df = pd.DataFrame({'date': ['3/10/2000', 'a/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'])

ParserError: Unknown string format: a/11/2000

In [33]:
df['date'] = pd.to_datetime(df['date'], errors='ignore')
df

,date,value
0,3/10/2000,2
1,a/11/2000,3
2,3/12/2000,4


In [34]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df

,date,value
0,2000-03-10,2
1,NaT,3
2,2000-03-12,4


# 2. Monte uma data e hora de várias colunas

to_datetime()também pode ser usado para montar um datetime de várias colunas. As chaves (rótulo das colunas) podem ser abreviações comuns como ['ano', 'mês', 'dia', 'minuto', 'segundo', 'ms', 'nós', 'ns']) ou plurais das mesmas .

In [35]:
df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5]})
df['date'] = pd.to_datetime(df)
df

,year,month,day,date
0,2015,2,4,2015-02-04
1,2016,3,5,2016-03-05


dt.year, dt.monthE dt.daysão os atributos do embutido para obter ano, mês e dia de Pandas datetime objeto.

Primeiro, vamos criar um DateFrame fictício e analisar DoB para data e hora.

In [36]:
df = pd.DataFrame({'name': ['Tom', 'Andy', 'Lucas'],
                 'DoB': ['08-05-1997', '04-28-1996', '12-16-1995']})
df['DoB'] = pd.to_datetime(df['DoB'])

In [37]:
df['year']= df['DoB'].dt.year
df['month']= df['DoB'].dt.month
df['day']= df['DoB'].dt.day
df

,name,DoB,year,month,day
0,Tom,1997-08-05,1997,8,5
1,Andy,1996-04-28,1996,4,28
2,Lucas,1995-12-16,1995,12,16


Da mesma forma, dt.week, dt.dayofweek, e dt.is_leap_yearsão os atributos do embutido para obter a semana do ano, o dia da semana, e ano bissexto.

In [38]:
df['week_of_year'] = df['DoB'].dt.week
df['day_of_week'] = df['DoB'].dt.dayofweek
df['is_leap_year'] = df['DoB'].dt.is_leap_year
df

c:\temp/ipykernel_12384/3969492849.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df['DoB'].dt.week


,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year
0,Tom,1997-08-05,1997,8,5,32,1,False
1,Andy,1996-04-28,1996,4,28,17,6,True
2,Lucas,1995-12-16,1995,12,16,50,5,False


In [39]:
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 
df['day_of_week_name']=df['DoB'].dt.weekday.map(dw_mapping)
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday


A solução mais simples para obter a idade é subtraindo o ano:

In [40]:
today = pd.to_datetime('today')
df['age'] = today.year - df['DoB'].dt.year
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name,age
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday,25
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday,26
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday,27


In [41]:
# Year difference
today = pd.to_datetime('today')
diff_y = today.year - df['DoB'].dt.year

# Haven't had birthday
b_md = df['DoB'].apply(lambda x: (x.month,x.day) )
no_birthday = b_md > (today.month,today.day)
df['age'] = diff_y - no_birthday

df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name,age
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday,24
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday,25
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday,26


Uma solução comum para selecionar dados por data é usar um maks booleano. Por exemplo

In [42]:
condition = (df['date'] > start_date) & (df['date'] <= end_date)
df.loc[condition]

KeyError: 'date'

Se você for fazer muitas seleções por data, seria mais rápido definir a coluna de data como o índice primeiro, para aproveitar as vantagens da otimização integrada do Pandas. Em seguida, você pode selecionar dados por data usando df.loc[start_date:end_date]. Vamos dar uma olhada em um exemplo de conjunto de dados city_sales.csv, que tem 1.795.144 linhas de dados

In [43]:
df = pd.read_csv('city_sales.csv',parse_dates=['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795144 entries, 0 to 1795143
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   date    datetime64[ns]
 1   num     int64         
 2   city    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 41.1+ MB


In [44]:
df = df.set_index(['date'])
df

,num,city
date,,
2015-01-01 09:00:00,4,London
2015-01-01 09:01:00,4,London
2015-01-01 09:02:00,3,London
2015-01-01 09:03:00,3,London
2015-01-01 09:04:00,3,London
...,...,...
2019-01-31 15:56:00,3,Cambridge
2019-01-31 15:57:00,3,Cambridge
2019-01-31 15:58:00,3,Cambridge


Digamos que gostaríamos de selecionar todos os dados no ano de 2018

In [45]:
df.loc['2018']

,num,city
date,,
2018-01-01 09:00:00,2,London
2018-01-01 09:01:00,1,London
2018-01-01 09:02:00,3,London
2018-01-01 09:03:00,3,London
2018-01-01 09:04:00,3,London
...,...,...
2018-12-31 15:56:00,4,Cambridge
2018-12-31 15:57:00,2,Cambridge
2018-12-31 15:58:00,3,Cambridge


Obtenha o número total em 2018

In [46]:
df.loc['2018','num'].sum()

1231190

In [47]:
df['2018'].groupby('city').sum()

c:\temp/ipykernel_12384/1368259670.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df['2018'].groupby('city').sum()


,num
city,
Cambridge,308428
Durham,307965
London,307431
Oxford,307366


Para selecionar dados com um mês específico, por exemplo, maio de 2018

In [48]:
df.loc['2018-5']

,num,city
date,,
2018-05-01 09:00:00,1,London
2018-05-01 09:01:00,4,London
2018-05-01 09:02:00,3,London
2018-05-01 09:03:00,2,London
2018-05-01 09:04:00,3,London
...,...,...
2018-05-31 15:56:00,3,Cambridge
2018-05-31 15:57:00,4,Cambridge
2018-05-31 15:58:00,2,Cambridge


In [49]:
df.loc['2018-5-1']

,num,city
date,,
2018-05-01 09:00:00,1,London
2018-05-01 09:01:00,4,London
2018-05-01 09:02:00,3,London
2018-05-01 09:03:00,2,London
2018-05-01 09:04:00,3,London
...,...,...
2018-05-01 15:56:00,2,Cambridge
2018-05-01 15:57:00,3,Cambridge
2018-05-01 15:58:00,3,Cambridge


Para selecionar dados entre duas datas, você pode usar df.loc[start_date:end_date]Por exemplo:

Selecione dados entre 2016 e 2018

In [50]:
df.loc['2016' : '2018']

c:\temp/ipykernel_12384/380412996.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df.loc['2016' : '2018']


,num,city
date,,
2016-01-01 09:00:00,4,London
2016-01-01 09:01:00,3,London
2016-01-01 09:02:00,4,London
2016-01-01 09:03:00,4,London
2016-01-01 09:04:00,2,London
...,...,...
2018-12-31 15:56:00,4,Cambridge
2018-12-31 15:57:00,2,Cambridge
2018-12-31 15:58:00,3,Cambridge


In [51]:
df.loc['2018-5-2 10' : '2018-5-2 11' ]

,num,city
date,,
2018-05-02 10:00:00,2,London
2018-05-02 10:01:00,3,London
2018-05-02 10:02:00,4,London
2018-05-02 10:03:00,4,London
2018-05-02 10:04:00,4,London
...,...,...
2018-05-02 11:55:00,3,Cambridge
2018-05-02 11:56:00,3,Cambridge
2018-05-02 11:57:00,4,Cambridge


In [52]:
df.loc['2018-5-2 10:30' : '2018-5-2 10:45' ]

,num,city
date,,
2018-05-02 10:30:00,3,London
2018-05-02 10:31:00,3,London
2018-05-02 10:32:00,1,London
2018-05-02 10:33:00,3,London
2018-05-02 10:34:00,3,London
...,...,...
2018-05-02 10:41:00,3,Cambridge
2018-05-02 10:42:00,3,Cambridge
2018-05-02 10:43:00,3,Cambridge


In [53]:
df.between_time('10:30','10:45')

,num,city
date,,
2015-01-01 10:30:00,4,London
2015-01-01 10:31:00,3,London
2015-01-01 10:32:00,3,London
2015-01-01 10:33:00,3,London
2015-01-01 10:34:00,4,London
...,...,...
2019-01-31 10:41:00,3,Cambridge
2019-01-31 10:42:00,3,Cambridge
2019-01-31 10:43:00,1,Cambridge


Freqüentemente, precisamos calcular estatísticas de janela, como uma média móvel ou uma soma móvel.

Vamos calcular a soma contínua em um período de 3 janelas e, em seguida, dar uma olhada nas 5 primeiras linhas.

In [54]:
df['rolling_sum'] = df.rolling(3).sum()
df.head()

,num,city,rolling_sum
date,,,
2015-01-01 09:00:00,4,London,NaN
2015-01-01 09:01:00,4,London,NaN
2015-01-01 09:02:00,3,London,11.0
2015-01-01 09:03:00,3,London,10.0
2015-01-01 09:04:00,3,London,9.0


In [55]:
df['rolling_sum_backfilled'] = df['rolling_sum'].fillna(method='backfill')
df.head()

,num,city,rolling_sum,rolling_sum_backfilled
date,,,,
2015-01-01 09:00:00,4,London,NaN,11.0
2015-01-01 09:01:00,4,London,NaN,11.0
2015-01-01 09:02:00,3,London,11.0,11.0
2015-01-01 09:03:00,3,London,10.0,10.0
2015-01-01 09:04:00,3,London,9.0,9.0
